# Interpretable Machine Learning
## Exercise Sheet 6: LIME
## This exercise sheet covers lecture 6 on LIME
Sophie Langbein (langbein@leibniz-bips.de)<br>
Pegah Golchian (golchian@leibniz-bips.de)
<hr style="border:1.5px solid gray"> </hr>

# Implementing LIME 

LIME, which stands for "Local Interpretable Model-Agnostic Explanations," is an interpretable machine learning technique used to explain the predictions of complex models. LIME provides insight into how a machine learning model arrived at a particular prediction by approximating the model's behavior in the local vicinity of a specific data point or instance.

In the following, you are guided to implement LIME to interpret machine learning models, specifically a support vector machine (SVM). For simplicity and the purpose of visualization we use only two (numeric) input features and explore LIME on a multiclass classification problem. We are considering the [wheat seeds dataset](https://archive.ics.uci.edu/dataset/236/seeds) from the UCI machine learning repository, which collects data for the purpose of classifying three different types of wheat Kernels (Kama = 0, Rosa = 1 and Canadian = 2),  and from it the `Area` and `Perimeter` features.  

**a)** Inspect the following implemented helper functions `get_grid()`, `plot_grid()` and `plot_points_in_grid()`

The function `get_grid()` prepares data to visualize the feature space. It creates a N × N grid, and every point in this grid is associated with a value. This value is obtained by the model’s predict method. It's designed for visualizing the behavior of a machine learning model over a grid of input values.

In [46]:
import sys
import os
import random
sys.path.insert(0, ".")

import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn import tree
from math import sqrt
import pandas as pd

def get_grid(model, dataset, points_per_feature = 50):
    """
    Retrieve grid data for plotting a two-dimensional graph with `points_per_feature` for each axis.
    The space is created by the hyperparameters' lower and upper values. Only the first two input
    labels are used.

    Parameters:
        model: A classifier that has a predict method. This is the machine learning model you want to evaluate.
        dataset (pd.dataframe): An input dataset of interest in the form of a pandas dataframe.
        points_per_feature: The number of points to generate for each feature dimension (default is 50).

    Returns:
        u (np.ndarray): An array containing the x-axis points, with a shape of (`points_per_feature`,).
        v (np.ndarray):  An array containing the y-axis points, with a shape of (`points_per_feature`,).
        z (np.ndarray): An array of values with a shape of (`points_per_feature`, `points_per_feature`). These values correspond to the model's predictions over the grid of input points.
    """

    labels = dataset.columns.tolist()  # extracts input labels from the provided dataset.

    x1 = np.linspace(min(dataset[labels[0]]),
                     max(dataset[labels[0]]), points_per_feature) # create linearly spaced array x1 for the first input feature, using the lower and upper bounds of the first hyperparameter
    x2 = np.linspace(min(dataset[labels[0]]),
                     max(dataset[labels[0]]), points_per_feature) # create linearly spaced array x2 for the second input feature, using the lower and upper bounds of the second hyperparameter

    X = [] 
    for x in itertools.product(x1, x2):
        X.append(x) # create a list X containing all possible combinations of the x1 and x2 values using itertools.product, these combinations represent the grid points

    X = np.array(X) # convert list X into a NumPy array X
    y = model.predict(X) # predicts the corresponding output values for the grid points using the provided model

    # Reshape all x
    u = x1
    v = x2
    z = y.reshape(points_per_feature, points_per_feature).T # reshapes the predicted y values to create the final z array

    return u, v, z

The function `plot_grid()`, visualizes the prediction surface. It is used to create a plot with a color grid overlay. It visualizesg the output of the get_grid function described earlier. 

In [36]:
def plot_grid(u, v, z, labels, title=None, embedded=False):
    """
    Uses the grid data to add a color grid to the plot.

    Parameters:
        u (np.ndarray):  An array containing x-axis points, with a shape of (N,).
        v (np.ndarray): An array containing y-axis points, with a shape of (N,).
        z (np.ndarray): An array of color values with a shape of (N, N).
        labels (list): A list containing labels for the x and y axes.
        embedded (bool): A boolean that determines whether a new figure should be created for the plot (default is False).

    Returns: 
        plt (matplotlib.pyplot or utils.styled_plot.plt): Plot with applied color grid.
    """

    if not embedded:
        plt.figure() # If embedded is False, a new figure is created using plt.figure(), this allows to start a new plot, especially when creating multiple plots in the same script or notebook

    plt.xlabel(labels[0]) # x-axis label is set to 'labels[0]'
    plt.ylabel(labels[1]) # x-axis label is set to 'labels[1]'
    plt.title(title) # title of the plot is set to 'title'

    plt.pcolormesh(u, v, z, cmap='viridis', shading='auto') # plt.pcolormesh() function is used to create the color grid based on the u, v, and z data
    plt.colorbar() # colorbar is added to the plot using plt.colorbar() to provide a reference for the color values
    plt.grid(alpha=0) # grid lines are turned off (made transparent) using plt.grid(alpha=0).

    return plt

The created plot is an input to the function `plot_points_in_grid()`, which adds scatter points to the existing plot.

In [37]:
def plot_points_in_grid(plt, Z=[], y=[], weights=None, colors={}, x_interest=None, size=8):
    """
    Given a plot, add scatter points from `Z` and `x_interest`.

    Parameters:
        plt (matplotlib.pyplot or utils.styled_plot.plt): Plot with color grid, representing the plot to which scatter points will be added
        Z (np.ndarray): Points with shape (?, 2) which should be added to the plot.
        y (np.ndarray): Target values with shape (?,), of the points added to the plot, determines the colouring of points.
        weights (np.ndarray): Normalized weights with shape (?,), determine the size of points in the plot.
        colors (dict): A dictionary that maps target values to colors. It returns the color for a given target value.
        x_interest (np.ndarray): Single point with shape (2,) whose prediciton we want to explain. If None (default) no point is added.
        size (int): Default size of the markers/points. Default is 8.
    """

    w = 1 # w is initialized with a value of 1, which represents the default weight for the size of the points

    for y_ in list(set(y)): # iterate through unique target values in y
        idx = np.where(y == y_)[0] # find the indices of the points in Z that have the current target value y_

        color = "black" 
        if y_ in colors:
            color = colors[y_] # If the target value exists in the colors dictionary, it uses the specified color; otherwise, it defaults to "black."

        if weights is not None:
            w = weights[idx] # If weights are provided (not None), it assigns the weights for the current target to the variable w

        plt.scatter(Z[idx, 0], Z[idx, 1], c=color, s=w*size, label=y_) # add scatter points to the plot.

    if x_interest is not None:
        plt.scatter([x_interest[0]], [x_interest[1]],
                    c='red', s=size, label="POI") # If x_interest is not None, it adds a red scatter point to the plot to represent the single point of interest (POI)

**b)** The first implementation task is to sample points, which are later used to train the local surrogate model. Complete `sample_points()` by randomly sampling from a uniform distribution. For sampling from the uniform distribution, consider the lower and upper bounds from the input datapoints.

**Solution:**

The function `sample_points()` is used to generate random sample points for the first two features in a dataset, typically for the purpose of visualizing model behavior over a range of input values

In [39]:
def sample_points(model, dataset, num_points, seed = 0):
    """
    Samples points for the two first features.

    Parameters:
        model: A classifier that has a predict method, this is the machine learning model for which you want to sample points.
        dataset (pd.dataframe): An input dataset of interest in the form of a pandas dataframe 
        num_points (int): An integer specifying how many points should be sampled.
        seed (int): An integer used as the seed for the random number generator.

    Returns:
        Z (np.ndarray): Sampled data points with shape (num_points, 2). These are two-dimensional input points.
        y (np.ndarray): Target values with shape (num_points,). These values are the model's predictions for the sampled data points.
    """

    return Z, y

**c)** Weight Points

Given a selected point $\mathbf{x}$ and the sampled points $Z$ from the previous task, we now want to weight the points.
Use the following equation with $d(x,z) = \sqrt{(x-z)^2}$ as Euclidean distance and $\sigma$ the kernel width to calculate the weight of a single point $\mathbf{z} \in Z$:

$$
\phi_{\mathbf{x}}(\mathbf{z}) = \exp(−d(\mathbf{x}, \mathbf{z})/\sigma^2)
$$

To make plotting easier later on, the weights should be normalized between zero and one. Finally, return the normalized weights in `weight_points()`.


**Solution:**

The function `weight_points()` calculates weights for a set of points based on their distances to a single point of interest (`x_interest`). The weights are calculated using an exponential kernel function. 

In [40]:
def weight_points(x_interest, Z, kernel_width=0.2):
    """
    For every z in `Z` returns a weight depending on the distance to `x_interest`.

    Parameters:
        x_interest (np.ndarray): Single point with shape (2,) whose prediction we want to explain.
        Z (np.ndarray): Points with shape (?, 2). These are the data points for which you want to calculate weights based on their distances to x_interest.
        kernel_width (float): A floating-point value representing the kernel width. It is used to calculate the distance according to an exponential kernel function. The default value is 0.2.

    Returns:
        weights (np.ndarray): An array of normalized weights with shape (?,). These weights represent the importance or influence of each data point in Z based on its distance from x_interest.
    """

    return weights

**d)** Fit Local Surrogate Model

Finally, fit a decision tree using the training data and the weights. Return the fitted tree in the function
`fit_explainer_model()`.


**Solution:**

The function `fit_explainer_model()` fits a decision tree regressor model to explain or approximate the relationship between input points (`Z`) and target values (`y`).

In [41]:
def fit_explainer_model(Z, y, weights=None, seed=0):
    """
    Fits a decision tree.

    Parameters:
        Z (np.ndarray): Points with shape (?, 2), used to fit surrogate model.
        y (np.ndarray): Target values with shape (?,). These are the values you want to explain or approximate.
        weights (np.ndarray): Normalized weights with shape (?,). These weights represent the importance of each data point in Z. If not provided, the weights are not used.
        seed (int): Seed for the decision tree.

    Returns:
        model (DecisionTreeRegressor): The fitted explainer model, which is a decision tree regressor.
    """

    return explainer_model

**e)** Now we want to assemble all the functions written above to implement LIME. In a first step, import the data from `wheat_seeds.csv`. 

- Drop all rows that contain `NA` values
- Drop all feature columns except `Area` and `Perimeter`. The outcome is recorded in the `Type` column.
- Create training and testing sets
- Fit a support vector machine to the training dataset, with the `gamma` parameter set to `auto`

**f)** Use the settings given below to plot the decision surface of the SVM model. Select a meaningful point of interest. 

Hint: Use the `get_grid()` and `plot_grid()` functions

**Solution:**

In [78]:
# Settings
x_interest = np.array([?, ?]) # set your own point of interest
points_per_feature = 50
n_points = 1000
labels = dataset.columns.tolist() 
colors = {
        0: "purple",
        1: "green",
        2: "orange",
    }

**g)** Generate sample points around the two features`Area` and `Perimenter` using the `sample_points()` function. Then plot the SVM decision surface and the Sampled Points using the `plot_grid()`and the `plot_points_in_grid()` functions. 

**h)** Weight the sampled points according to their distance to the point of interest `x_interest` using the `weight_points()` function. Then plot the SVM decision surface together with the weighted sampled points again using the `plot_grid()`and the `plot_points_in_grid()` functions. 

**i)** Now fit a local surrogate model in the form of a decision tree using the `fit_explainer_model()`function.

**j)** Plot the decision surface of the surrogate decision tree model, including the point of interest and the weighted sampled points comparison purposes. Is the local surrogate model a good fit? 

<hr style="border:1.5px solid gray"> </hr>

# Application of LIME

For a practical application of LIME, consider the Seoul Bike Sharing dataset, which was taken from the UCI machine learning repository https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand. The dataset contains count of public bicycles rented per hour in the Seoul Bike Sharing System, with corresponding weather data and holiday information. 

**a)** Import the Seoul Bike Sharing dataset from `SeoulBikeData.csv`. Use one hot encoding to encode all categorical features. Then split the data into training and test set. 

**b)** For an application of LIME, fit a gradient boosting regression model to the preprocessed data. Calculate the $R^2$ on the test set to evaluate the model fit. 

**c)** Use the LIME from the `lime` package to give local explanations of the first and the 11th instance of the test data. Use a local ridge regression (default) as a surrogate model and discretize the data. Then visualize the results using an adequate plot using using 8 features. Interpret the results. Why could the local explanations be problematic?

**d)** Now replace the default ridge regression with a decision tree as a surrogate model. Plot the results for the first instance and compare them to the results from **c)**. What do you notice?

**e)** Now use KernelSHAP to generate an explanation for the first instance. Visualize the results in a force plot. Compare them to the results to the LIME for the first instance. What do you notice? Why is this happening? In your opinion, which method is more useful SHAP or LIME? Briefly explain your reasons.  

<hr style="border:1.5px solid gray"> </hr>